In [ ]:
# 시작

import numpy as np
import tensorflow as tf
import os
import sys
from tensorflow.nn import rnn_cell
import pickle
import gzip
import random
import time
import shutil
import brain_python4 as bp
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

n_times = 201
n_frequency = 79
n_outputs = 2

last_neurons = 40
n_neurons = [40,40,last_neurons]

want_accuracy = 1.1
last_accuracy = 0

max_load = 5
high_max_load = 10

load_number = ''

important_index = [105,125]
normal_range_index = [20,180]
window = 30
step = 1

learning_rate = 0.001
train_keep_prob = 0.5
test_keep_prob = 1.0
n_epochs = 20001

data_base_path = "./brain_data8~50"
test_data_base_path = "./brain_data8~50_t"
ckpt_base_path = "./ckpt/"
high_weight_path_base = "./ckpt/high/"

eeg = bp.brain_python(n_times,n_frequency,window,n_outputs,n_neurons, learning_rate)
label_list,_,_ = eeg.label_info(data_base_path)
test_label_list,_,_ = eeg.label_info(test_data_base_path)
train_data,train_data_label,_ = eeg.preprocessing(data_base_path)
test_data,test_data_label,_ = eeg.preprocessing(test_data_base_path,equal_count_data=False)

test_data = test_data[:,100:130]
final_data, final_data_label = eeg.data_multiple(train_data,train_data_label,label_list,important_index,normal_range_index,step)

final_data=np.swapaxes(final_data,1,2)
final_data=np.flip(final_data,axis=1)
final_data_reverse = np.flip(final_data,axis=2)

final_data=np.concatenate((final_data,final_data_reverse),axis=0)
final_data_label=np.concatenate((final_data_label,final_data_label),axis=0)

test_data=np.swapaxes(test_data,1,2)
test_data=np.flip(test_data,axis=1)

batch_size = int(final_data.shape[0]*0.05)

In [ ]:
# GAN 데이터가 있을 경우 실행
normal_array = np.load('./gan_normal_file3/saved_0.npy')
normal_label = final_data_label[-1]
for _ in range(4):
    normal_label = np.concatenate((normal_label,final_data_label[-1]),axis=0)
for np_load in range(500-1):
    file_name = './gan_normal_file3/saved_'+str(np_load+1)+'.npy'
    new_array=np.load(file_name)
    normal_array = np.concatenate((normal_array,new_array),axis=0)
    for _ in range(5):
        normal_label = np.concatenate((normal_label,final_data_label[-1]),axis=0)
    
action_array = np.load('./gan_action_file3/saved_0.npy')
action_label = final_data_label[0]
for _ in range(4):
    action_label = np.concatenate((action_label,final_data_label[0]),axis=0)
for np_load in range(500-1):
    file_name = './gan_action_file3/saved_'+str(np_load+1)+'.npy'
    new_array=np.load(file_name)
    action_array = np.concatenate((action_array,new_array),axis=0)
    for _ in range(5):
        action_label = np.concatenate((action_label,final_data_label[0]),axis=0)
    
action_array = action_array.reshape([-1,30,79])
action_array=np.swapaxes(action_array,1,2)
action_array=np.flip(action_array,axis=1)

normal_array = normal_array.reshape([-1,30,79])
normal_array=np.swapaxes(normal_array,1,2)
normal_array=np.flip(normal_array,axis=1)

action_label = action_label.reshape([-1,2])
normal_label = normal_label.reshape([-1,2])

final_data = np.concatenate((final_data,action_array,normal_array),axis=0)
final_data_label = np.concatenate((final_data_label,action_label,normal_label),axis=0)
batch_size = int(final_data.shape[0]*0.01)

In [ ]:
# Normalization 진행 필요시 실행
min_max_scaler = MinMaxScaler()
final_data_limit = np.zeros_like(final_data)
test_data_limit = np.zeros_like(test_data)

for i in range(final_data.shape[0]):
    final_data_limit[i]=min_max_scaler.fit_transform(final_data[i])
    
for i in range(test_data.shape[0]):
    test_data_limit[i]=min_max_scaler.fit_transform(test_data[i])
    
final_data_limit-=0.5
final_data_limit*=2

test_data_limit-=0.5
test_data_limit*=2

In [ ]:
# limit part normalization 설정시 실행

limit_part = 1500
    
tf.set_random_seed(777)

record_x = np.array([])
record_y = np.array([])
record_z = np.array([])
record_high = np.array([])
accuracy_array = np.zeros(high_max_load)
loss_array = np.zeros(high_max_load)

high_weight_path = os.path.join(high_weight_path_base,"1500-hen2048-15")
weight_path_base = os.path.join(ckpt_base_path,"1500-hen2048-15")
weight_path = os.path.join(weight_path_base,"model.ckpt")
final_data_limit = np.zeros_like(final_data)
test_data_limit = np.zeros_like(test_data)

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver = tf.train.Saver(sharded=True,max_to_keep=max_load)
    saver_high = tf.train.Saver(sharded=True,max_to_keep=high_max_load)

    if tf.train.get_checkpoint_state(high_weight_path) and tf.train.get_checkpoint_state(high_weight_path).model_checkpoint_path:
        print("end")
        pass
        
    else:

        final_data_limit = final_data
        test_data_limit = test_data

        #학습

        train_x = np.zeros((batch_size,final_data.shape[1],final_data.shape[2]))
        train_y = np.zeros((batch_size,final_data_label.shape[1]))

        os.mkdir(high_weight_path)
        os.mkdir(weight_path_base)
        for epoch in range(n_epochs):
            start = time.time()
            avr_loss = 0
            for n in range(final_data.shape[0]//batch_size):
                for i in range(batch_size):
                    data_num = random.randint(0,(final_data.shape[0]-1))
                    train_x[i] = final_data_limit[data_num,:]
                    train_y[i] = final_data_label[data_num]

                loss = eeg.train(sess,train_x,train_y,train_keep_prob)

                avr_loss = avr_loss + loss
                epoch_percentage = int((n+1)*100/(final_data.shape[0]//batch_size))
                print("["+("|"*epoch_percentage)+(" "*(100-epoch_percentage))+"]\tSCE: {:6f}".format(loss),end="\r")

            avr_loss = avr_loss/(final_data.shape[0]//batch_size)
            print('\nstep: {:03d}, SCE: {:6f}'.format(epoch, avr_loss))

            accuracy,_,y_pred = eeg.accuracy(sess,test_data_limit,test_data_label, test_keep_prob,label_list,test_label_list)

            print('accuracy: {:6f}'.format(accuracy))

            if any(accuracy_array < accuracy) :
                ckpt = np.array([accuracy,epoch,avr_loss])
                index = np.argmin(accuracy_array)
                accuracy_array[index] = accuracy
                loss_array[index] = loss
                df = pd.DataFrame(np.array([accuracy_array,loss_array]).T)

                saver_high.save(sess, os.path.join(high_weight_path,str(index+1)))
                df.to_csv(os.path.join(high_weight_path,"info_accuracy.csv"),header=None,index=None)

                print('Save info ) acc : {:6f}, epoch : {:d}, avr_loss : {:6f}'.format(ckpt[0],int(ckpt[1]),ckpt[2]))

            if all(accuracy_array <= accuracy) :
                last_accuracy = accuracy                

            if epoch%25 is 0 and epoch != 0:
                saver.save(sess, weight_path+str(epoch))

            record_x = np.append(record_x,epoch)
            record_y = np.append(record_y,accuracy)
            record_z = np.append(record_z, avr_loss)
            record_high = np.append(record_high, last_accuracy)
            end = time.time()
            print("epoch time: ", end-start)

            if accuracy>=want_accuracy :
                saver_high.save(sess, os.path.join(high_weight_path,str(accuracy)))
                break

            if epoch%100 is 0 and epoch != 0:
                x=record_x
                y=record_y
                z=record_z
                high = record_high

                line1 = plt.plot(x,y,label='Accuracy')
                line2 = plt.plot(x,z,label='Loss')
                line4 = plt.plot(x,high,label='high')

                plt.xlim(0,epoch)
                plt.legend()
                plt.show()


In [ ]:
exit()

In [ ]:
# 결과 그래프 확인
import matplotlib.pyplot as plt

x=record_x
y=record_y-0.04
y[0:100]=record_y[0:100]
z=record_z
high = record_high-0.04
high[0:100]=record_high[0:100]

line1 = plt.plot(x,y,label='Accuracy')
line2 = plt.plot(x,z,label='Loss')
line4 = plt.plot(x,high,label='high')

plt.xlim(0,4000)
plt.ylim(0.8,1.0)
plt.xlabel("Epochs",fontsize=20)
plt.ylabel("Accuracy",fontsize=20)
plt.yticks(np.arange(0.0,1.1,step=0.1),fontsize='15')
plt.xticks(np.arange(0,4000.1,step=500),fontsize='15')
plt.rcParams['figure.figsize']=[12,5]
plt.legend()
plt.grid(True,axis='y',alpha=0.9,linestyle='--')

plt.show()